# Création et résolution de labyrinthes

Références :
Wikipedia pour l'algorithme d'exploration exhaustive
"Mazes for Programmers" de Jamis Buck, qui traite de différents problèmes liés aux labyrinthes, et propose des implémentations en Ruby.

# 1°) Introduction

## 1_1 Présentation

L'objectif de ce mini-projet est de voir différents algorithmes permettant de construire des labyrinthes, puis une méthode de résolution
Nous nous restreindrons à des labyrinthes dits parfaits, c'est à dire des labyrinthes où chaque cellule peut joindre une autre par un chemin unique.
A l'inverse, les labyrinthes pouvant contenir des ilôts, des boucles ou des cellules inaccessibles sont dits imparfaits, et ne seront pas étudiés dans ce projet.

## 1_2 Grille

Avant de construire un labyrinthe, nous allons commencer par construire une grille.
Nous avons pour cela défini plusieurs classes :
la classe UneCellule, permettant de créer une cellule de la grille. Cette cellule sera caractérisée par sa position dans la grille, et possède 4 "bords" appelés murs : le mur Nord, le mur Sud, le mur Est et le mur Ouest.
la classe Grille, qui est un tableau de nx lignes et nxcolonnes de cellules.

In [96]:
########
class UneCellule:
    """
    définition d'une cellule
    """
    
    def __init__(self,x, y):
        """
        créer une cellule positionnée en (x=ligne, y=colonne)
        """
        
        self.x = x
        self.y = y
        #les murs sont dans l'ordre : N, S, E, O. 
        #la valeur est à True si un mur est présent, False sinon
        self.murs = {'N': True, 'S': True, 'E': True, 'O': True}


########      
class Grille :
    """
    construction d'une grille de cellules
    """
    
    def __init__(self, nx, ny):
        """
        construction d'une grille de dimension (nx, ny)
        """
        
        self.nx = nx
        self.ny = ny
        self.cadrillage = []
        for i in range(nx):
            GrilleLigne=[]
            for j in range(ny):
                GrilleLigne.append(UneCellule(i,j))
            self.cadrillage.append(GrilleLigne)
        
        
    def cellule(self, x, y):
        """
        retourne la cellule de la grille de position (x=ligne, y=colonne)
        """
        
        return self.cadrillage[x][y]
    
    def __str__(self):
        """
        retourne une chaine représentant le labyrinthe
        Attention : seuls les murs Est et Sud d'une cellule sont représentés
        """
        laby_lignes = ['+---' * self.ny+'+']
        
        for x in range(self.nx):
            laby_l = ['|']
            for y in range(self.ny):
                if self.cadrillage[x][y].murs['E']:
                    laby_l.append('   |')
                else:
                    laby_l.append('    ')
            laby_lignes.append(''.join(laby_l))
            laby_l = ['+']
            for y in range(self.ny):
                if self.cadrillage[x][y].murs['S']:
                    laby_l.append('---+')
                else:
                    laby_l.append('   +')
            laby_lignes.append(''.join(laby_l))
        return '\n'.join(laby_lignes)

## 1_3 Utilisation des classes

Afin de mieux comprendre les classes ci-dessus, voyons quelques exemples.

In [97]:
# Contruction d'une grille de 4 lignes par 6 colonnes
test = Grille(4,6)
print(test)

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


In [98]:
# Accès aux dimensions de la grille
print(test.nx, test.ny)

4 6


In [99]:
# Accès à une cellule de la grille
cell =test.cellule(2,3)

In [100]:
# Accès à la position d'une cellule (sous forme (ligne, colonne))
print(cell.x, cell.y)

2 3


In [101]:
# Accès aux murs d'une cellule
cell.murs

{'N': True, 'S': True, 'E': True, 'O': True}

In [102]:
# Accès au mur Est d'une cellule
cell.murs['E']

True

## 1_4 Premier exercice

Reprenons la grille test précédente, et commençons par détruire quelques murs.

In [103]:
# Détruisons le mur Est de la cellule 'cell' positionnée en (2,3)
cell.murs['E']=False
print(test)

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |       |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


On remarque que le mur Est de la cellule (2,3) est bien détruit. Mais pour autant ... le mur Ouest de la cellule (2,4) est encore intact !!

In [104]:
test.cellule(2,4).murs['O']

True

Cela est bien problèmatique, car en terme de représentation du labyrinthe, le mur Est de (2,3) est bien le même mur est que le mur Ouest de (2,4) !
Plus problèmatique, encore : reconstruisons le mur Est de (2,3), détruisons le mur Ouest de (2,4), et visualisons le résultat :

In [105]:
test.cellule(2,3).murs['E']=True
test.cellule(2,4).murs['O']=False
print(test)

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


Comme vous pouvez le voir, le mur Ouest de (2,4) est bien détruit, et pourtant cela n'apparaît pas dans la visualisation de la grille.
Cela est du à la façon de construire la représentation de la grille. Par commodité, seuls les murs Est et Sud d'une cellule sont traçés. Cela permet de ne pas afficher de 'doubles murs', mais en contrepartie le lien qui existe entre deux cellules de part et d'autres d'un même mur est perdu.
Nous aurons le même problème avec par exemple le mur Nord de (2,3) et le mur Sud de (1,3).

In [106]:
# casser le mur Nord de (2,3) passera inaperçu
test = Grille(4,6)
test.cellule(2,3).murs['N']=False
print(test)

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


In [107]:
# si on veut voir disparaitre le mur, il faut détruire le mur Sud de (1,3)
test.cellule(1,3).murs['S']=False
print(test)

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+   +---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


### Exercice :

Afin de palier à ce problème, créer une fonction effaceMur(grille, coord, orientation) qui pour une grille grille :
efface le mur orientation (c-à-d 'N', 'S', 'E' ou 'O') de la cellule de position coord.
efface aussi le mur correspondant de la cellule voisine.

In [108]:
def effaceMur(grille, coord, orientation)->None:
    grille.cellule(coord[0],coord[1]).murs[orientation]=False

    if orientation == 'S' and coord[0]==grille.nx-1:
        rien = 'rien'
    elif orientation == 'N' and coord[0]==0:
        rien = 'rien'
    elif orientation == 'E' and coord[1]==grille.ny-1:
        rien = 'rien'
    elif orientation == 'O' and coord[1]==0:
        rien = 'rien'
    else:
        if orientation == 'S':
            grille.cellule(coord[0]+1,coord[1]).murs['N'] = False
        elif orientation == 'N':
            grille.cellule(coord[0]-1,coord[1]).murs['S'] = False
        elif orientation == 'O':
            grille.cellule(coord[0],coord[1]-1).murs['E'] = False
        elif orientation == 'E':
            grille.cellule(coord[0],coord[1]+1).murs['O'] = False
            
    return None

Exemple :

In [109]:
test = Grille(4,6)

# on efface le mur Ouest de la cellule (3,2)
effaceMur(test,(3,2),'O')

# affichage de la cellule modifiée
print(test.cellule(3,2).murs)

# affichage de la cellule 'voisine', ici la cellule (3,1)
print(test.cellule(3,1).murs)

# affichage de la grille
print(test)

{'N': True, 'S': True, 'E': True, 'O': False}
{'N': True, 'S': True, 'E': False, 'O': True}
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |       |   |   |   |
+---+---+---+---+---+---+


In [110]:
test = Grille(4,6)

# on efface le mur Nord de la cellule (2,1)
effaceMur(test,(2,1),'N')

# affichage de la cellule modifiée
print(test.cellule(2,1).murs)

# affichage de la cellule 'voisine', ici la cellule (1,1)
print(test.cellule(1,1).murs)

# affichage de la grille
print(test)

{'N': False, 'S': True, 'E': True, 'O': True}
{'N': True, 'S': False, 'E': True, 'O': True}
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+   +---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


In [111]:
test = Grille(4,6)

# on efface le mur Est de la cellule (0,5)
effaceMur(test,(0,5),'E')

# affichage de la cellule modifiée
print(test.cellule(0,5).murs)

# Attention : cette cellule n'a pas de cellule voisine située à sa droite, donc il ne doit pas y avoir de modifications supplémentaires !

# affichage de la grille
print(test)

{'N': True, 'S': True, 'E': False, 'O': True}
+---+---+---+---+---+---+
|   |   |   |   |   |    
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


# 2°) Construction de labyrinthes parfaits

Nous allons voir dans cette partie plusieurs algorithmes permettant de créer des labyrinthes parfaits. Nous ne nous occuperons pas de placer les entrées et sorties du labyrinthes, qui n'influent pas sur la construction.
Ainsi, les murs extérieurs de la grille ne seront jamais détruits.

## 2_1 Algorithme de l'arbre binaire

Nous allons construire notre premier labyrinthe, à l'aide d'un algorithme très simple. En voici sa description :
on part d'une grille où tous les murs existent.
on parcourt une à une toutes les cellules de la grille (en commençant en haut à gauche, c'est à dire en (0,0)).
pour chaque cellule ainsi parcourue, on détruit aléatoirement soit le mur Sud soit le mur Est.
On fera attention que si une cellule est située sur le bord droit de la grille, c'est le mur Sud qui sera détruit.
De même, si une cellule est située sur le bord du bas de la grille, c'est focément le mur Est qui sera détruit.
on ne détruira aucun mur de la cellule située tout en bas à droite.

### Exercice :

In [112]:
from random import *

Construire une fonction arbreBinaire(grille) qui réalise l'algorithme précédent.

In [113]:
def arbreBinaire(grille):
    for j in range(grille.nx):
        for i in range(grille.ny):
            orien = choice(['E', 'S'])
            if i == grille.ny-1:
                orien = 'S'
            if j == grille.nx-1:
                orien = 'E'
            if j == grille.nx-1 and i == grille.ny-1:
                orien = ''
            effaceMur(grille, (j, i), orien)
    return grille

Exemple :

In [114]:
AB = Grille(4,6)
print(arbreBinaire(AB))

+---+---+---+---+---+---+
|               |       |
+---+---+---+   +---+   +
|           |       |   |
+---+---+   +---+   +   +
|       |           |   |
+---+   +---+---+   +   +
|                       |
+---+---+---+---+---+---+


Essayez de construire plusieurs labyrinthes à l'aide de cette méthode. Qu'ont en commun tous ces labyrinthes ?

ils ont tous leur ligne du bas et leur colonne de droite ouverts

## 2_2 Algorithme sidewinder

Cet algorithme simple est très similaire à l'algorithme précédent. Le labyrinthe est généré une ligne à la fois : pour chaque cellule de cette ligne, on décide aléatoirement s'il faut détruire le mur Est ou pas.
Si oui, alors on détruit le mur Est.
Sinon, on considère le passage horizontal qui vient d'être terminé, formé par la cellule actuelle et toutes les cellules à gauche qui ont creusé des passages menant à celle-ci. On choisit alors aléatoirement une cellule le long de ce passage et on détruit son mur Sud.

### Exercice :

Construire une fonction sidewinder(grille) qui réalise l'algorithme précédent.

In [115]:
def sidewinder(grille):
    timerDeb = 0
    timerFin = 0
    for i in range(grille.nx-1):
        for j in range(grille.ny):
            alea = randint(0,1)
            if (j < grille.ny):
                if (alea == 0) and (j < grille.ny-1):
                    effaceMur(grille,(i,j),'E')
                    timerFin += 1
                elif (j == grille.ny-1) and timerFin == timerDeb:
                    effaceMur(grille,(i,j),'S')
                else:
                    distance = timerFin-timerDeb
                    newAlea=randint(0,distance)
                    effaceMur(grille,(i,j-newAlea),'S')
                    timerDeb = timerFin

    #traitement de la dernière ligne
    for i in range(grille.ny-1):
        effaceMur(grille,(grille.nx-1,i),'E')

    return grille

Exemple :

In [116]:
SW = Grille(4,6)
print(sidewinder(SW))

+---+---+---+---+---+---+
|       |   |       |   |
+   +---+   +---+   +   +
|       |   |       |   |
+---+   +   +---+   +   +
|       |       |   |   |
+   +---+---+   +   +   +
|                       |
+---+---+---+---+---+---+


Essayez de construire plusieurs labyrinthes à l'aide de cette méthode. Qu'ont en commun tous ces labyrinthes ?

ils ont tous leur ligne du bas ouvert

## 2_3 Algorithme d'exploration exhaustive

Cet algorithme s'apparente au parcours en profondeur d'arbre vu en cours. Voici la description de cet algorithme sur wikipedia:
Exploration Exhaustive
L'utilisation de la récursivité n'étant pas au programme, nous allons comme en TP de graphes utiliser une Pile.

### Exercice :

Construire une fonction explorationE(grille) qui réalise l'algorithme précédent.

In [117]:
def explorationE(grille):
    h = grille.nx
    l = grille.ny
    Visite = []
    for i in range(h):
        petitTab = []
        for j in range(l):
            petitTab += [0]
        Visite += [petitTab]
    Pile = [(randint(0,h-1),randint(0,l-1))]
    Visite[Pile[0][0]][Pile[0][1]] = 1
    
    while Pile != []:
        
        voisins = []
        if  Pile[0][0] < h-1 and Visite[Pile[0][0]+1][Pile[0][1]] == 0:
            voisins += [(Pile[0][0]+1,Pile[0][1],'S')]

        if  Pile[0][0] > 0 and Visite[Pile[0][0]-1][Pile[0][1]] == 0:
            voisins += [(Pile[0][0]-1,Pile[0][1],'N')]

        if  Pile[0][1] < l-1 and Visite[Pile[0][0]][Pile[0][1]+1] == 0:
            voisins += [(Pile[0][0],Pile[0][1]+1,'E')]

        if  Pile[0][1] > 0 and Visite[Pile[0][0]][Pile[0][1]-1] == 0:
            voisins += [(Pile[0][0],Pile[0][1]-1,'O')]
            
        if voisins != []:
            alea = randint(0,len(voisins)-1)
            destru = (voisins[alea])
            effaceMur(grille,(Pile[0][0],Pile[0][1]),destru[2])
            Visite[destru[0]][destru[1]] = 1
            Pile = [(destru[0],destru[1])] + Pile
        else:
            Pile.pop(0)
            
    return grille

Exemple :

In [131]:
test = Grille(4,6)
Exp = explorationE(test)
print(Exp)

+---+---+---+---+---+---+
|           |           |
+   +---+---+   +---+   +
|   |       |   |   |   |
+   +   +   +   +   +---+
|   |   |       |       |
+   +   +---+---+---+   +
|                       |
+---+---+---+---+---+---+


# 3°) Résolution d'un labyrinthe parfait

## 3_1 Introduction

Nous allons proposer maintenant une résolution de labyrinthe, par une version simplifiée de l'algorithme de Dijkstra.
Pour cela, nous ferons la convention que l'entrée du labyrinthe est la cellule en haut à gauche et que la sortie est la cellule située en bas à droite.
Nous dirons que deux cellules sont attenantes ssi il n'y a pas de murs entre elles.

### Exercice :

Construire une fonction attenantes(labyrinthe,coord) qui retourne les cellules attenantes à la cellule de position coord.

In [119]:
def attenantes(grille,coord):
    res = []

    if  coord[0] < grille.nx-1 and grille.cellule(coord[0],coord[1]).murs['S']==False:
        res += [(coord[0]+1,coord[1])]

    if  coord[0] > 0 and grille.cellule(coord[0],coord[1]).murs['N']==False:
        res += [(coord[0]-1,coord[1])]

    if  coord[1] < grille.ny-1 and grille.cellule(coord[0],coord[1]).murs['E']==False:
        res += [(coord[0],coord[1]+1)]

    if  coord[1] > 0 and grille.cellule(coord[0],coord[1]).murs['O']==False:
        res += [(coord[0],coord[1]-1)]
        
    return res

Exemple :

In [120]:
laby = explorationE(Grille(4,6))
print(laby)

+---+---+---+---+---+---+
|               |   |   |
+   +   +---+---+   +   +
|   |   |       |   |   |
+   +---+   +   +   +   +
|           |       |   |
+   +---+---+---+---+   +
|                       |
+---+---+---+---+---+---+


In [121]:
print(attenantes(laby,(1,3)))

[(2, 3), (1, 2)]


In [122]:
print(attenantes(laby,(1,1)))

[(0, 1)]


## 3_2 Calcul des distances depuis (0,0)

In [123]:
from math import inf

L'objectif de cette partie est d'obtenir un tableau (ou une matrice ...) notée D, où :
D[i,j] = nombre de cases nécéssaires pour se rendre de la case (0,0) à la case (i,j).

Nous allons cette fois utiliser une File.

### Exercice

Construire une fonction distance(labyrinthe) qui retourne la matrice D pour un labyrinthe donné.

In [124]:
def distance(labyrinthe):
    D = [[inf] * labyrinthe.ny for _ in range(labyrinthe.nx)]
    #Ce tableau contiendra les distances depuis (0,0), et peut être initialisé à l'infini

    File = []

    #Initialisation :
    D[0][0] = 0
    File += [(0,0)]
    
    while File != []:
        ij = File[-1]
        lesatt = attenantes(laby, ij)
        for k in range (len (lesatt)):
            if D[lesatt[k][0]][lesatt[k][1]] == inf:
                File = [lesatt[k]] + File
                D[lesatt[k][0]][lesatt[k][1]] = D[ij[0]][ij[1]]+1
        del File[-1]

    return D

In [125]:
print(laby,"\n")
print(distance(laby))

+---+---+---+---+---+---+
|               |   |   |
+   +   +---+---+   +   +
|   |   |       |   |   |
+   +---+   +   +   +   +
|           |       |   |
+   +---+---+---+---+   +
|                       |
+---+---+---+---+---+---+ 

[[0, 1, 2, 3, 10, 11], [1, 2, 5, 6, 9, 10], [2, 3, 4, 7, 8, 9], [3, 4, 5, 6, 7, 8]]


## 3_3 Résolution

Pour trouver la solution au labyrinthe, il "suffit" de partir de la cellule en bas à droite, et de remonter le chemin dans D en suivant l'ordre décroissant.

### Exercice :

Construire une fonction resolution(labyrinthe) qui retourne le chemin permettant de résoudre le labyrinthe.

In [126]:
def resolution(labyrinthe):

    dist = distance(labyrinthe)
    pos = (labyrinthe.nx-1,labyrinthe.ny-1)
    retour = [pos]

    for i in range(labyrinthe.nx-1):
        for j in range(labyrinthe.ny-1):
            voisins = attenantes(labyrinthe,(pos[0],pos[1]))
            compteur = 0
            while compteur != -1 and compteur < 5:
                if dist[pos[0]][pos[1]] > 0 and (dist[voisins[compteur][0]][voisins[compteur][1]] == dist[pos[0]][pos[1]]-1):

                    pos = (voisins[compteur][0],voisins[compteur][1])
                    retour = [pos] + retour
                    
                    compteur = -1
                else:
                    compteur += 1
    return retour

Exemple :

In [127]:
print(laby,"\n")
print(resolution(laby))

+---+---+---+---+---+---+
|               |   |   |
+   +   +---+---+   +   +
|   |   |       |   |   |
+   +---+   +   +   +   +
|           |       |   |
+   +---+---+---+---+   +
|                       |
+---+---+---+---+---+---+ 

[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
